# Map Wrapper Development for Scipy Differential Equation

Proof of concept notebook for first Scipy, since it only requires map not the o

Error_function is a pass function.
Our "map" takes care of initiating the whole neuron group, calculating the errors and returing the value.

```
params, fits, error = fit_traces(model = model, input_var = 'v', output_var = 'I',\
                                 input = input_traces, output = output_traces,
                                 dt = 0.1*ms, g = [1*nS, 30*nS], E = [-20*mV,100*mV],
                                 tol = 1e-6)
```

In [1]:
import numpy as np
import multiprocessing as mp

from scipy.optimize import differential_evolution, rosen
from scipy.optimize._differentialevolution import DifferentialEvolutionSolver

from nevergrad.optimization import optimizerlib
from concurrent import futures
from nevergrad import instrumentation as inst

In [2]:
from brian2 import *
from brian2.equations.equations import (DIFFERENTIAL_EQUATION, Equations,
                                        SingleEquation, PARAMETER)
from brian2.input import TimedArray
from brian2 import NeuronGroup, StateMonitor, store, restore, run, defaultclock, second, Quantity
from brian2.stateupdaters.base import StateUpdateMethod



#### Artificial Data

In [3]:
params = np.array([
 [ 1.80869973e-08,  2.50218013e-02],
 [ 1.88373085e-08,  9.89559934e-02], 
 [ 1.88373085e-08,  9.89559934e-02], 
])


In [4]:
params2 = np.array([[ 1.12735759e-08,  8.81556360e-02],
 [ 9.04858783e-09,  8.03343489e-02],
 [ 1.60635907e-08,  1.59710651e-03],
 [ 1.66254878e-09,  2.95479385e-02],
 [ 1.80869973e-08,  2.50218013e-02],])

### Input / Output

In [5]:
input_traces = zeros((10,1))*volt
for i in range(1):
    input_traces[1:,i]=i*10*mV

In [6]:
# Create target current traces
output_traces = 10*nS*input_traces

In [7]:
input = input_traces
output = output_traces

### Parameters 

In [8]:
input_var = 'v'
output_var = 'I'

parameter_names = {'g', 'E'}
method = ('linear', 'exponential_euler', 'euler')
t_start = 0*second
popsize, _ = np.shape(params)
dt = 0.1 *ms
defaultclock.dt = dt

### Calculation for inner parameters

In [9]:
Nsteps, Ntraces = input_traces.shape
# N = popsize * len(parameter_names)
N = popsize
duration = Nsteps*dt

In [10]:
N

3

In [11]:
Ntraces

1

### Setup The Model for Optimization

In [12]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [13]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

INFO       No numerical integration method specified, using method 'linear' (took 0.01s). [brian2.stateupdaters.base.method_choice]


### Required Model Operations

In [14]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [15]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

In [16]:
neurons = NeuronGroup(Ntraces*N, model, method = method)
neurons.namespace['input_var'] = input_traces
neurons.namespace['output_var'] = output_traces
neurons.namespace['t_start'] = t_start
neurons.namespace['Ntraces'] = Ntraces

# Record error
neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                      when='end')

# Add the code doing the numerical integration
neurons.run_regularly(state_update_code, when='groups')

store()


In [17]:
neurons

NeuronGroup(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='neurongroup')

### Dev Map Optimization 

### Create the Dictonaries of Parameters

In [18]:
def parameters_dict(params):
    d = dict()
    for name, value in zip(parameter_names, params.T):
        d[name] = value
            
    return d

In [19]:
d = parameters_dict(params)
d

{'g': array([1.80869973e-08, 1.88373085e-08, 1.88373085e-08]),
 'E': array([0.0250218 , 0.09895599, 0.09895599])}

In [20]:
neurons.get_states()

{'N': array(3),
 'i': array([0, 1, 2], dtype=int32),
 't': 0. * second,
 'dt': 100. * usecond,
 't_in_timesteps': array(0),
 'g': array([0., 0., 0.]) * siemens,
 'E': array([0., 0., 0.]) * volt,
 'total_error': array([0., 0., 0.]) * amp2}

In [21]:
restore()
neurons.set_states(d, units=False)
neurons.get_states()

{'N': array(3),
 'i': array([0, 1, 2], dtype=int32),
 't': 0. * second,
 'dt': 100. * usecond,
 't_in_timesteps': array(0),
 'g': array([18.0869973, 18.8373085, 18.8373085]) * nsiemens,
 'E': array([25.0218013, 98.9559934, 98.9559934]) * mvolt,
 'total_error': array([0., 0., 0.]) * amp2}

In [22]:
run(duration, namespace = {})

In [23]:
e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
e = mean(e.reshape((N,Ntraces)),axis=1)
array(e)

array([2.04818928e-19, 3.47473674e-18, 3.47473674e-18])

## All of the above into one hard-coded function

In [24]:
start_scope()

In [25]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [26]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

In [27]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [28]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

In [29]:
def calc_error(params):
    print(params)
    popsize, _ = np.shape(params)
    N = popsize

#     neurons = NeuronGroup(Ntraces*N, model, method = method)
    neurons = NeuronGroup(N, model, method = method)
    neurons.namespace['input_var'] = input_traces
    neurons.namespace['output_var'] = output_traces
    neurons.namespace['t_start'] = t_start
    neurons.namespace['Ntraces'] = Ntraces

    # Record error
    neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                          when='end')

    # Add the code doing the numerical integration
    neurons.run_regularly(state_update_code, when='groups')

    d = parameters_dict(params)
    neurons.set_states(d, units=False)
    run(duration, namespace = {})

    e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
    e = mean(e.reshape((N,Ntraces)),axis=1)
    return array(e).T

In [30]:
calc_error(params2)

[[1.12735759e-08 8.81556360e-02]
 [9.04858783e-09 8.03343489e-02]
 [1.60635907e-08 1.59710651e-03]
 [1.66254878e-09 2.95479385e-02]
 [1.80869973e-08 2.50218013e-02]]


array([9.87696585e-19, 5.28401654e-19, 6.58192637e-22, 2.41325473e-21,
       2.04818928e-19])

## Try The Same with Differential Evolution from Scipy

In [31]:
start_scope()

In [32]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [33]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

In [34]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [35]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

### Ask and Tel with calc_error Function

In [37]:
arg1 = inst.var.Array(1).bounded(-5, 5).asscalar()
arg2 = inst.var.Array(1).bounded(0, 10).asscalar()
instrum = inst.Instrumentation(arg1, arg2)

In [38]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000, num_workers=2)

In [78]:
n_samples = 10
params = []

for _ in range(n_samples):
    cand = optim.ask()
    params.append(list(cand.args))
    
params

[[-0.8977993930323029, 1.75035854412906],
 [1.5586276417088412, 1.9948889456620322],
 [-2.076484183775389, 6.54745031743287],
 [-1.3415947720034402, 2.0297533842720465],
 [2.777495918792162, 6.626468992797089],
 [-1.8431581146083993, 1.2534851794991684],
 [-1.4489875924285205, 4.596787602221065],
 [3.44086608463614, 6.3906924474229],
 [0.21281001269756605, 7.56818173844709],
 [-3.5748448199950933, 4.804784164678763]]

In [88]:
errors = calc_error(np.array(params))
errors

[[-0.89779939  1.75035854]
 [ 1.55862764  1.99488895]
 [-2.07648418  6.54745032]
 [-1.34159477  2.02975338]
 [ 2.77749592  6.62646899]
 [-1.84315811  1.25348518]
 [-1.44898759  4.5967876 ]
 [ 3.44086608  6.39069245]
 [ 0.21281001  7.56818174]
 [-3.57484482  4.80478416]]


In [92]:
for i, candidate in enumerate(candidates):
    value = errors[i]
    optim.tell(candidate, value)
    
    print(candidate, value)


Candidate(args=(2.5492653744515947, 8.217405039191796), kwargs={}) 2.469520596280959
Candidate(args=(-3.04019557703552, 2.575170654002903), kwargs={}) 9.667678414161967
Candidate(args=(-0.4896946760281114, 7.663318262685749), kwargs={}) 184.84243385524766
Candidate(args=(3.7184965650971242, 7.643142903038951), kwargs={}) 7.415309167169591
Candidate(args=(2.752800009397278, 7.793754179187996), kwargs={}) 338.74367837860297
Candidate(args=(-0.2912003154746927, 2.278110616922909), kwargs={}) 5.337815914148098
Candidate(args=(2.5930302573689348, 6.551607796220621), kwargs={}) 44.36476730624581
Candidate(args=(0.5190581628111219, 4.514738007345433), kwargs={}) 483.5388534812149
Candidate(args=(-0.12594335770807727, 8.177455086246518), kwargs={}) 2.5939835650293435
Candidate(args=(3.1093787997319207, 7.468545172010227), kwargs={}) 295.02726666545146


In [98]:
ans = optim.provide_recommendation()
list(ans.args)

[2.5492653744515947, 8.217405039191796]

# Translate To Big Ugly Function

In [99]:
n_samples = 10
params = []



for _ in range(n_samples):
    cand = optim.ask()
    params.append(list(cand.args))

errors = calc_error(np.array(params))

for i, candidate in enumerate(candidates):
    value = errors[i]
    optim.tell(candidate, value)
    
ans = optim.provide_recommendation()

[[ 3.15074886  8.21740504]
 [ 1.87744565  2.57517065]
 [-0.48969468  7.92771073]
 [ 3.64127577  9.27582324]
 [ 3.54288754  7.83832004]
 [ 4.02645529  2.27811062]
 [ 2.59303026  8.41961438]
 [-0.1489514   7.0455189 ]
 [ 2.79560434  8.41136261]
 [ 3.96604687  3.4605173 ]]


[-3.04019557703552, 2.575170654002903]